In [1]:
%load_ext autoreload
%autoreload 2

%cd ..

/Users/rubenbroekx/Documents/Projects/agoro-field-boundary-detector


# Interference

In [2]:
import ee
import folium

from pathlib import Path

from src.agoro_field_boundary_detector import FieldBoundaryDetectorInterface
from src.agoro_field_boundary_detector.google_earth_engine import (
    start_session, 
    create_bounding_box, 
    to_polygon,
    NaipCollection,
    create_map,
    show_point,
    show_polygon,
    create_polygon,
)

In [3]:
start_session()

Enter verification code: 4/1AY0e-g4V1w8CfZBX_En2WOZdcHMOAyrTn1PhpMbOSzfsJ_mgI2_8_SsajbQ

Successfully saved authorization token.


In [21]:
# Specify the coordinate to query
LAT=39.6679328199836
LNG=-95.4287818841267

In [22]:
model = FieldBoundaryDetectorInterface(
    model_path=Path.cwd() / 'models/mask_rcnn',
    new_session=False,
)

In [23]:
# Find the field's boundary
boundary = model(lat=LAT, lng=LNG)
boundary = [(b,a) for a,b in boundary]
polygon = create_polygon([boundary])

In [24]:
# Create a map visualisation
mp = create_map(
    (LAT,LNG), 
    zoom=14,
)

# Create and display dataset
collection = NaipCollection(
    region=polygon,
)
mp.add_ee_layer(
    collection.get_image(),
    collection.get_vis_params(),
    'NAIP',
)

# Display supplementary information
mp = show_polygon(mp, polygon, tag="Field")
mp = show_point(mp, ee.Geometry.Point(LNG,LAT))

# Show map
mp.add_child(folium.LayerControl())
display(mp)